# Redis Product Catalog

## Product Catalog Use Case

In the last session several common Redis use cases were discussed. Now, lets consider using Redis as our primary database for a product catalog for a new online store. It will need to store product details including a name, description, vendor, price, main category and some images.

## Requirements:

* Product information stored in the database should include: name, description, vendor, price, category, images associated with that product.
* Ability to create/update/delete product details
* Ability to find product by ID
* Ability to find products in category X
* Ability to find product by its name or part of its name

## Step 3 - Logical Data Model:

The logical data model is separate from the DBMS being used.
It defines the structure of data elements and to set relationships between them.

* Product Image
    * Id: Number
    * Value: Binary

* Product
    * Id: Number
    * Name: String
    * Description: String
    * Vendor: String
    * Price: Number
    * Currency: String
    * MainCategory: Category (1)
    * Images: Image (0..n)
 
* Category
    * id: Number
    * Name: String
    * Products: Product (0..n)

In [1]:
# Lets make some sample products in a json:

In [3]:
# Product Image
product_image = {
"Id": 1,
"Value": 'binaryvalue'
}

In [4]:
# Product
product = {
    'Id': 1,
    'Name': 'ProductA',
    'Description': 'Chicken',
    'Vendor': 'Walmart',
    'Price': 10,
    'Currency': 'dollars',
    'MainCategory': 1, # mapped to category (Id)
    'Images': [1,2,3,4] # mapped to product_image (Id)
}

In [5]:
# Category
category = {
    "Id": 1,
    "Name": 'ProductA',
    "Products": [1,2,3] # mapped to product (Id)
}

## Step 4 - Convert to Physical Data Model for Redis:

How would you take this logical data model and turn it into a physical model in Redis?

What data structures would you use to meet the requirements listed above?

How to index and query the data?

Be ready to show via redis-cli (or Insight) how you stored the data and how you are able to find the product by ID, list all prodcuts in a given category and find a Product by full or partial name.

In [67]:
# product image: what should we store it as in redis?
product_image

{'Id': 1, 'Value': 'binaryvalue'}

In [74]:
# it looks like we can store it as a simple String (key value pair)
# we will need to fix the structure how how we deal with this though.

Id 1
Value binaryvalue


In [93]:
# lets make a product image data structure in python
def product_image_kv(product_id, value):
    product_image = {f"product_image:{product_id}":{
    'Value': value,
    }}
    return product_image

In [104]:
# lets create some values
import random

random.seed()
product_id = random.getrandbits(32)
value = 100

product_image = product_image_kv(product_id, value)
product_image

{'product_image:3355641040': {'Value': 100}}

In [172]:
# lets convert it to a redis data structure (a string, a key value)
for Id, product_dict in product_image.items():
    #print(Id, product_dict)
    for k,v in product_dict.items():
            #redis.set(Id,v)
            #print(k,v)
            redis.set(Id,v)
    #check it out
    print(Id)
    print(redis.get(Id))
    

product_image:3355641040
b'100'


In [107]:
# lets make the Category data model

In [109]:
print(category)

{'Id': 1, 'Name': 'ProductA', 'Products': [1, 2, 3]}


In [ ]:
# we will need to follow a similar process that we did with the product image.
# this time, we cant use a string data structure, we need a different one.
# a Hash is really the best option

# the challenge here is that we need to map in all the product ids.

In [110]:
def category_hash(category_id, name, products):
    category = {f"category:{category_id}":{
    'Name': name,
    'Products': products
    }}
    return category

In [128]:
import random

random.seed()
category_id = random.getrandbits(32)
name = 'Meat'
products = [1,2,5,6,7]

category = category_hash(category_id, name, products)
category

{'category:2423841559': {'Name': 'Meat', 'Products': [1, 2, 5, 6, 7]}}

In [171]:
# lets convert it to a redis data structure (a string, a key value)
for Id, category_dict in category.items():
    #print(Id, category_dict)
    for k,v in category_dict.items():
            #print(k,v)
            if k != 'Products':
                redis.hset(Id,k,v)
            else:
                #print(v)
                # add products to a new data structure with a set and a new id
                for i in v:
                    #print(i)
                    set_id_products = f"{Id}:products"
                    redisList = redis.sadd(set_id_products,i)
            #redis.hset(Id,k,v)
    #check it out
    print(Id)
    print(redis.hgetall(Id))
    print(set_id_products)
    print(redis.smembers(set_id_products))

category:2423841559
{b'Name': b'Meat'}
category:2423841559:products
{b'6', b'7', b'2', b'5', b'1'}


In [162]:
redis.flushdb()

True

In [148]:
redis.scan()

(0, [b'category:2423841559:products', b'category:2423841559'])

In [ ]:
# lets make a product data model

# we will use a hash function.

# the challenge here is the we need the image and category to fit into this model.

In [149]:
def product_hash(product_id, name, description, vendor, price, currency, mainCategory, images):
    product = {f"product:{product_id}":{
    'Name': name,
    'Description': description,
    'Vendor': vendor,
    'Price': price,
    'Currency': currency,
    'MainCategory': mainCategory, # mapped to category (Id)
    'Images':  images# mapped to product_image (Id)
    }}
    return product

In [159]:
import random

random.seed()
product_id = random.getrandbits(32)
name = 'ProductA'
description = 'Chicken'
vendor = 'CostCo'
price = 12
currency = 'dollars'
mainCategory = [1,5,67,8]
images = [1,2,3,4]

products = product_hash(product_id, name, description, vendor, price, currency, mainCategory, images)

In [170]:
# lets convert it to a redis data structure (a string, a key value)
for Id, product_dict in products.items():
    #print(Id, product_dict)
    for k,v in product_dict.items():
            #print(k,v)
            if k != 'MainCategory' and k != 'Images':
                redis.hset(Id,k,v)
            elif k == 'MainCategory':
                #print(v)
                # add products to a new data structure with a set and a new id
                for i in v:
                    #print(i)
                    set_id_maincat = f"{Id}:maincat"
                    redis.sadd(set_id_maincat,i)
            elif k == 'Images':
                #print(v)
                # add products to a new data structure with a set and a new id
                for i in v:
                    #print(i)
                    set_id_images = f"{Id}:images"
                    redis.sadd(set_id_images,i)
            #redis.hset(Id,k,v)
    #check it out
    print(Id)
    print(redis.hgetall(Id))
    print(set_id_maincat)
    print(redis.smembers(set_id_maincat))
    print(set_id_images)
    print(redis.smembers(set_id_images))

product:1900991853
{b'Name': b'ProductA', b'Description': b'Chicken', b'Vendor': b'CostCo', b'Price': b'12', b'Currency': b'dollars'}
product:1900991853:maincat
{b'67', b'8', b'5', b'1'}
product:1900991853:images
{b'4', b'3', b'2', b'1'}


In [81]:
with redis.pipeline() as pipe:
    for Id, product_dict in product.items():
        print(Id, product_dict)
        for k,v in product_dict.items():
            pipe.hset(Id,k,v)
    pipe.execute()

product:1326692461 {'Name': 'ProductA', 'Description': 'Chicken', 'Vendor': 'CostCo', 'Price': 12, 'Currency': 'dollars', 'MainCategory': 1, 'Images': 1}


In [83]:
print(redis.hgetall("product:1326692461"))

{b'Name': b'ProductA', b'Description': b'Chicken', b'Vendor': b'CostCo', b'Price': b'12', b'Currency': b'dollars', b'MainCategory': b'1', b'Images': b'1'}


In [75]:
redis.get('product-image:1')

In [10]:
for k,v in product.items():
    print(k,v)

Id 1
Name ProductA
Description Chicken
Vendor Walmart
Price 10
Currency dollars
MainCategory 1
Images [1, 2, 3, 4]


In [7]:
# import redis
import redis
import os

# connect redis to localhost and port
redis = redis.Redis(host='localhost', port=6379, db=0)

In [8]:
# simple commands to test
redis.set('foo','bar')
redis.get('foo')

b'bar'

In [48]:
import random

random.seed(444)
# create product dictionary generator.
# identify the product id as "product:(random int)"
# then add each products meta data.
product = {f"product:{random.getrandbits(32)}": i for i in (
    {
    'Name': 'ProductA',
    'Description': 'Chicken',
    'Vendor': 'CostCo',
    'Price': 10,
    'Currency': 'dollars',
    'MainCategory': 1, # mapped to category (Id)
    'Images': 1 # mapped to product_image (Id)
    },
    {
    'Name': 'ProductB',
    'Description': 'Pork',
    'Vendor': 'HEB',
    'Price': 10,
    'Currency': 'dollars',
    'MainCategory': 2, # mapped to category (Id)
    'Images': 1 # mapped to product_image (Id)
    },
    {
    'Name': 'ProductC',
    'Description': 'Duck',
    'Vendor': 'WholeFoods',
    'Price': 10,
    'Currency': 'dollars',
    'MainCategory': 4, # mapped to category (Id)
    'Images': 1 # mapped to product_image (Id)
    },)
}


In [49]:
product

{'product:1326692461': {'Name': 'ProductA', 'Description': 'Chicken', 'Vendor': 'CostCo', 'Price': 10, 'Currency': 'dollars', 'MainCategory': 1, 'Images': 1}, 'product:1236154736': {'Name': 'ProductB', 'Description': 'Pork', 'Vendor': 'HEB', 'Price': 10, 'Currency': 'dollars', 'MainCategory': 2, 'Images': 1}, 'product:56854717': {'Name': 'ProductC', 'Description': 'Duck', 'Vendor': 'WholeFoods', 'Price': 10, 'Currency': 'dollars', 'MainCategory': 4, 'Images': 1}}

In [65]:

with redis.pipeline() as pipe:
    for Id, product_dict in product.items():
        print(Id, product_dict)
        for k,v in product_dict.items():
            pipe.hset(Id,k,v)
    pipe.execute()

product:1326692461 {'Name': 'ProductA', 'Description': 'Chicken', 'Vendor': 'CostCo', 'Price': 10, 'Currency': 'dollars', 'MainCategory': 1, 'Images': 1}
product:1236154736 {'Name': 'ProductB', 'Description': 'Pork', 'Vendor': 'HEB', 'Price': 10, 'Currency': 'dollars', 'MainCategory': 2, 'Images': 1}
product:56854717 {'Name': 'ProductC', 'Description': 'Duck', 'Vendor': 'WholeFoods', 'Price': 10, 'Currency': 'dollars', 'MainCategory': 4, 'Images': 1}


In [66]:
print(redis.hgetall("product:56854717"))

{b'Name': b'ProductC', b'Description': b'Duck', b'Vendor': b'WholeFoods', b'Price': b'10', b'Currency': b'dollars', b'MainCategory': b'4', b'Images': b'1'}


In [64]:
redis.hdel("product:1326692461", 'Id')

1